### Data to Streamlit app: AI for BI


## From Customer Feedback to Actionable Insights 
### Using Multi-Agent CrewAI and Streamlit to Automate Reporting and Visualization 

- read the data and format it into CSV - save the result
- build the report as Markdown - save the result
- in the lefthand column of the st app display the report as markdown
- make a pandas dataframe in the st app from the CSV
- make a histogram chart from the dataframe and display it in the right hand column
- make a second histgram and display it below the first one   
- construct a st app with header and two columns using the previous results - save the app

In [1]:
from crewai import Agent, Task, Crew, LLM

llm = LLM(
    model="gpt-4o-mini",
    temperature=0.0
)

DEBUG = True

In [2]:
# files

fb_raw = "./data/shoes.md"

fb_csv = "./data/fb.csv"
report_md = "./data/report.md"
st_app = "./data/report.py"

Import the tools required to read and write files

In [3]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

### Agent and task to construct a data structure from the raw Markdown file of customer feedback

In [4]:

csv_agent = Agent(
        role="Extract, process data and record data",
        goal="""Extract data and organise as instructed. The result MUST be valid CSV.""",
        backstory="""You are a data processing agent""",
        tools=[file_read_tool],
        llm=llm,
    )

In [5]:
create_CSV = Task(
    description=""" 
                Analyse '{input_file}' the data provided - it is in Markdown format. 
                Your output should be in CSV format. Respond without using Markdown code fences.
                The data is about the range of items in an online shop.
                Following this is a set of messages from customers giving feedback about the shoes that they have purchased.
                Your task is to:
                   Create a structured file in CSV format that records a list of all customer feedback messages.
                   Ensure that string data is enclosed in quotes.
                   Each item in the list should have its columns populated as follows.
                                       
                        "Product": insert the name of the item, 
                        "Overall_Rating": insert the rating as given by customer, 
                        "Issue": insert any issues identified - if no issue can be identified write the word 'None', 
                        "Review": insert the customer message
                
                """,
    expected_output="A correctly formatted CSV data structure",
    agent=csv_agent,   
    tools=[file_read_tool]
)

In [6]:
add_sentiment = Task(
    description=""" 
                Analyse CSV data and calculate the sentiment of each message in
                the 'Review' column. Add a new column to the CSV that records that sentiment.
                Your output should be in CSV format. Respond without using Markdown code fences.              
                """,
    expected_output="A correctly formatted CSV data file",
    agent=csv_agent,   
    output_file=fb_csv,
    tools=[file_read_tool]
)

In [7]:
crew = Crew(
    agents=[csv_agent, csv_agent],
    tasks=[create_CSV, add_sentiment],
    verbose=DEBUG,
)
result1 = crew.kickoff(inputs={'input_file': fb_raw})

# Agent: Extract, process data and record data
## Task:  
                Analyse './data/shoes.md' the data provided - it is in Markdown format. 
                Your output should be in CSV format. Respond without using Markdown code fences.
                The data is about the range of items in an online shop.
                Following this is a set of messages from customers giving feedback about the shoes that they have purchased.
                Your task is to:
                   Create a structured file in CSV format that records a list of all customer feedback messages.
                   Ensure that string data is enclosed in quotes.
                   Each item in the list should have its columns populated as follows.
                                       
                        "Product": insert the name of the item, 
                        "Overall_Rating": insert the rating as given by customer, 
                        "Issue": insert any issues identified - if no is

### Report writing agent

In [8]:

# Define agent
report_generator = Agent(
        role="Compile and present results",
        goal="""Deliver a polished, structured report on customer satisfaction.
           """,
        backstory="""You are an agent, that generates clear, well-designed
                     and professional reports""",
        tools=[file_read_tool],
        llm=llm,
    )


In [9]:
create_report = Task(
    description="""
            Read the CSV data in '{csv_file}', create a summary report.
            The report that consolidates and summarizes the customer feedback,                
            it should be in Markdown file format (without ``` fencing). 
        
            The report should be structured as follows:

                # Product review report

                ### Summary

                Insert a Markdown table with an row for every product.
                The table header should look like this:
                
                | Product | Average Rating | Number of reviews | Positve | Neutral | Negative |

                The should be a row for every product like this:

                | insert the product name here 
                | insert the average of all the rating for this product 
                | insert total number of reviews
                | insert number of positive reviews 
                | insert number of neutral reviews 
                | insert number of negative reviews |

                ### Insights

                #### Best performers

                insert a short report on the products with the best reviews

                #### Underperformers

                insert a short report on the products that are underperforming

                #### Issues

                insert a short report on what steps need to be taken to improve products and sales
    """,
    expected_output="""A Markdown report file""",
    agent=report_generator,
    output_file = report_md,
    tools=[file_read_tool]
)

In [10]:
crew = Crew(
    agents=[report_generator],
    tasks=[create_report],
    verbose=DEBUG,
)
result2 = crew.kickoff(inputs={'input_file':fb_raw, 'csv_file': fb_csv})

2024-12-18 15:18:26,726 - 17840 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Compile and present results
## Task: 
            Read the CSV data in './data/fb.csv', create a summary report.
            The report that consolidates and summarizes the customer feedback,                
            it should be in Markdown file format (without ``` fencing). 
        
            The report should be structured as follows:

                # Product review report

                ### Summary

                Insert a Markdown table with an row for every product.
                The table header should look like this:
                
                | Product | Average Rating | Number of reviews | Positve | Neutral | Negative |

                The should be a row for every product like this:

                | insert the product name here 
                | insert the average of all the rating for this product 
                | insert total number of reviews
                | insert number of positive reviews 
                | insert number of neutral r

## Streamlit app

In [11]:
# Define agent
app_generator = Agent(
        role="Create or modify a Streamlit program",
        goal="""To deliver a valid, Streamlit program  in Python, with meaningful variable and function names
                Ensure that the output strictly conforms to Python syntax.
                Do not include ``` fencing.

           """,
        backstory="""You are an agent that generates clear, well-designed
                     Streamlit programs""",
        tools=[file_writer_tool, file_read_tool],
        llm=llm,
    )


In [12]:
create_app = Task(
    description="""
            Create a Streamlit app as follows:
            - set the display to wide format
            - include the pandas and ploty express libraries
            - create a pandas dataframe called "df" from the csv file {csv_file} include all fields
            Note that st.cache is deprecated, use st.cach_data instead.

            Don't save the program to a file.

    """,
    expected_output="""A valid and syntactically correct Python program""",
    agent=app_generator,
    tools=[file_read_tool]
)

In [13]:
add_content = Task(
    description="""
            Modify the Streamlit code as follows:
            - create two tabs, the first named "report_tab" the second named "messages_tab"
            - in "messages_tab" load the dataframe, "df", in a st.table 
            - in report_tab create two columns of equal width called "report_column" and "chart_column"
            - in the column "report_column" read the report from {report_file} and display it in st.markdown
            - in the "chart_column" draw a bar chart of 'Product' over 'Overall_Rating'
            - in the column "chart_column" draw a histogram of 'Sentiment' with the bars colored by 'Product'

    """,
    expected_output="""A valid and syntactically correct Python program""",
    output_file = st_app,
    agent=app_generator,
    tools=[file_read_tool]   
)         

In [14]:

crew = Crew(
    agents=[app_generator, app_generator],
    tasks=[create_app, add_content],
    verbose=DEBUG,
)
result3 = crew.kickoff(inputs={'csv_file': fb_csv ,'report_file': report_md})

2024-12-18 15:18:47,684 - 17840 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Create or modify a Streamlit program
## Task: 
            Create a Streamlit app as follows:
            - set the display to wide format
            - include the pandas and ploty express libraries
            - create a pandas dataframe called "df" from the csv file ./data/fb.csv include all fields
            Note that st.cache is deprecated, use st.cach_data instead.

            Don't save the program to a file.

    


# Agent: Create or modify a Streamlit program
## Final Answer: 
import streamlit as st
import pandas as pd
import plotly.express as px

# Set the display to wide format
st.set_page_config(layout="wide")

# Load the data from the CSV file
@st.cache_data
def load_data():
    df = pd.read_csv('./data/fb.csv')
    return df

# Create the DataFrame
df = load_data()

# Display the DataFrame
st.write(df)


# Agent: Create or modify a Streamlit program
## Task: 
            Modify the Streamlit code as follows:
            - create two tabs, the first named "repo

It seems that the following error occurs when the task description is too complex. So, better to break them up into smaller tasks.

## Tool Output:
Error: the Action Input is not a valid key, value dictionary.
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.


![](image.png)